In [13]:
import pandas as pd
import nltk
import string

In [49]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/pepino/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/pepino/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/pepino/nltk_data...


True

In [4]:
fake_data = pd.read_csv('data/Fake.csv')
fake_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB


In [8]:
fake_data.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [10]:
fake_data['subject'].value_counts()

subject
News               9050
politics           6841
left-news          4459
Government News    1570
US_News             783
Middle-east         778
Name: count, dtype: int64

In [11]:
text_data = fake_data['text']

In [16]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [35]:
ex_text = text_data[0]
tt = ex_text.maketrans({x:'' for x in string.punctuation})
ex_text = ex_text.translate(tt)
ex_text = ex_text.lower()

In [45]:
tokenized_text = nltk.tokenize.word_tokenize(ex_text)

In [46]:
stopwords = nltk.corpus.stopwords.words('english')
removed_stopwords = [word for word in tokenized_text if word not in stopwords]

In [47]:
removed_stopwords

['donald',
 'trump',
 'wish',
 'americans',
 'happy',
 'new',
 'year',
 'leave',
 'instead',
 'give',
 'shout',
 'enemies',
 'haters',
 'dishonest',
 'fake',
 'news',
 'media',
 'former',
 'reality',
 'show',
 'star',
 'one',
 'job',
 'country',
 'rapidly',
 'grows',
 'stronger',
 'smarter',
 'want',
 'wish',
 'friends',
 'supporters',
 'enemies',
 'haters',
 'even',
 'dishonest',
 'fake',
 'news',
 'media',
 'happy',
 'healthy',
 'new',
 'year',
 'president',
 'angry',
 'pants',
 'tweeted',
 '2018',
 'great',
 'year',
 'america',
 'country',
 'rapidly',
 'grows',
 'stronger',
 'smarter',
 'want',
 'wish',
 'friends',
 'supporters',
 'enemies',
 'haters',
 'even',
 'dishonest',
 'fake',
 'news',
 'media',
 'happy',
 'healthy',
 'new',
 'year',
 '2018',
 'great',
 'year',
 'america',
 'donald',
 'j',
 'trump',
 'realdonaldtrump',
 'december',
 '31',
 '2017trump',
 'tweet',
 'went',
 'welll',
 'expectwhat',
 'kind',
 'president',
 'sends',
 'new',
 'year',
 'greeting',
 'like',
 'despica

In [50]:
lemmatizer = nltk.stem.WordNetLemmatizer()
lemmatized = [lemmatizer.lemmatize(word) for word in removed_stopwords]
lemmatized

['donald',
 'trump',
 'wish',
 'american',
 'happy',
 'new',
 'year',
 'leave',
 'instead',
 'give',
 'shout',
 'enemy',
 'hater',
 'dishonest',
 'fake',
 'news',
 'medium',
 'former',
 'reality',
 'show',
 'star',
 'one',
 'job',
 'country',
 'rapidly',
 'grows',
 'stronger',
 'smarter',
 'want',
 'wish',
 'friend',
 'supporter',
 'enemy',
 'hater',
 'even',
 'dishonest',
 'fake',
 'news',
 'medium',
 'happy',
 'healthy',
 'new',
 'year',
 'president',
 'angry',
 'pant',
 'tweeted',
 '2018',
 'great',
 'year',
 'america',
 'country',
 'rapidly',
 'grows',
 'stronger',
 'smarter',
 'want',
 'wish',
 'friend',
 'supporter',
 'enemy',
 'hater',
 'even',
 'dishonest',
 'fake',
 'news',
 'medium',
 'happy',
 'healthy',
 'new',
 'year',
 '2018',
 'great',
 'year',
 'america',
 'donald',
 'j',
 'trump',
 'realdonaldtrump',
 'december',
 '31',
 '2017trump',
 'tweet',
 'went',
 'welll',
 'expectwhat',
 'kind',
 'president',
 'sends',
 'new',
 'year',
 'greeting',
 'like',
 'despicable',
 'pett